<a href="https://colab.research.google.com/github/betsenara/UCSD-ML-Engineering-AI/blob/main/Student_MLE_MiniProject_Recommendation_EnginesSolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Mini Project: Recommendation Engines

Recommendation engines are algorithms designed to provide personalized suggestions or recommendations to users. These systems analyze user behavior, preferences, and interactions with items (products, movies, music, articles, etc.) to predict and offer items that users are likely to be interested in. Recommendation engines play a crucial role in enhancing user experience, driving engagement, and increasing conversion rates in various applications, including e-commerce, entertainment, content platforms, and more.

There are generally two approaches taken in collaborative filtering and content-based recommendation engines:

**1. Collaborative Filtering:**
Collaborative Filtering is a popular approach to building recommendation systems that leverages the collective behavior of users to make personalized recommendations. It is based on the idea that users who have agreed in the past will likely agree in the future. There are two main types of collaborative filtering:

- **User-based Collaborative Filtering:** This method finds users similar to the target user based on their past interactions (e.g., ratings or purchases). It then recommends items that similar users have liked but the target user has not interacted with yet.

- **Item-based Collaborative Filtering:** In this approach, the system identifies similar items based on user interactions. It recommends items that are similar to the ones the target user has already liked or interacted with.

Collaborative filtering does not require any explicit information about items but relies on the similarity between users or items. It is effective in capturing complex patterns and can provide serendipitous recommendations. However, it suffers from the cold-start problem (i.e., difficulty in recommending to new users or items with no interactions) and scalability challenges in large datasets.

**2. Content-Based Recommendation:**
Content-based recommendation is an alternative approach to building recommendation systems that focuses on the attributes or features of items and users. It leverages the characteristics of items to make recommendations. The key steps involved in content-based recommendation are:

- **Feature Extraction:** For each item, relevant features are extracted. For movies, these features could be genre, director, actors, and plot summary.

- **User Profile:** A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.

- **Similarity Calculation:** The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity or Euclidean distance.

- **Recommendation:** Items that are most similar to the user profile are recommended to the user.

Content-based recommendation systems are less affected by the cold-start problem as they can still recommend items based on their features. They are also more interpretable as they rely on item attributes. However, they may miss out on providing serendipitous recommendations and can be limited by the quality of feature extraction and user profiles.

**Choosing Between Collaborative Filtering and Content-Based:**
Both collaborative filtering and content-based approaches have their strengths and weaknesses. The choice between them depends on the specific requirements of the recommendation system, the type of data available, and the user base. Hybrid approaches that combine collaborative filtering and content-based techniques are also common, aiming to leverage the strengths of both methods and mitigate their weaknesses.

In this mini-project, you'll be building both content based and collaborative filtering engines for the [MovieLens 25M dataset](https://grouplens.org/datasets/movielens/25m/). The MovieLens 25M dataset is one of the most widely used and popular datasets for building and evaluating recommendation systems. It is provided by the GroupLens Research project, which collects and studies datasets related to movie ratings and recommendations. The MovieLens 25M dataset contains movie ratings and other related information contributed by users of the MovieLens website.

**Dataset Details:**
- **Size:** The dataset contains approximately 25 million movie ratings.
- **Users:** It includes ratings from over 162,000 users.
- **Movies:** The dataset consists of ratings for more than 62,000 movies.
- **Ratings:** The ratings are provided on a scale of 1 to 5, where 1 is the lowest rating and 5 is the highest.
- **Timestamps:** Each rating is associated with a timestamp, indicating when the rating was given.

**Data Files:**
The dataset is usually split into three CSV files:

1. **movies.csv:** Contains information about movies, including the movie ID, title, genres, and release year.
   - Columns: movieId, title, genres

2. **ratings.csv:** Contains movie ratings provided by users, including the user ID, movie ID, rating, and timestamp.
   - Columns: userId, movieId, rating, timestamp

3. **tags.csv:** Contains user-generated tags for movies, including the user ID, movie ID, tag, and timestamp.
   - Columns: userId, movieId, tag, timestamp

First, import all the libraries you'll need.

In [51]:
import zipfile
import numpy as np
import pandas as pd
from urllib.request import urlretrieve
from sklearn.metrics.pairwise import cosine_similarity

Next, download the relevant components of the MoveLens dataset. Note, these instructions are roughly based on the colab [here](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/recommendation-systems/recommendation-systems.ipynb?utm_source=ss-recommendation-systems&utm_campaign=colab-external&utm_medium=referral&utm_content=recommendation-systems#scrollTo=O3bcgduFo4s6).

In [52]:
print("Downloading movielens data...")

urlretrieve('http://files.grouplens.org/datasets/movielens/ml-100k.zip', 'movielens.zip')
zip_ref = zipfile.ZipFile('movielens.zip', 'r')
zip_ref.extractall()
print("Done. Dataset contains:")
print(zip_ref.read('ml-100k/u.info'))

ratings_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(
    'ml-100k/u.data', sep='\t', names=ratings_cols, encoding='latin-1')

# The movies file contains a binary feature for each genre.
genre_cols = [
    "genre_unknown", "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]
movies_cols = [
    'movie_id', 'title', 'release_date', "video_release_date", "imdb_url"
] + genre_cols
movies = pd.read_csv(
    'ml-100k/u.item', sep='|', names=movies_cols, encoding='latin-1')

Done. Dataset contains:
b'943 users\n1682 items\n100000 ratings\n'


Before doing any kind of machine learning, it's always good to familiarize yourself with the datasets you'lll be working with.

Here are your tasks:

1. Spend some time familiarizing yourself with both the `movies` and `ratings` dataframes. How many unique user ids are present? How many unique movies are there?
2. Create a new dataframe that merges the `movies` and `ratings` tables on 'movie_id'. Only keep the 'user_id', 'title', 'rating' fields in this new dataframe.

In [53]:
# Spend some time familiarizing yourself with both the movies and ratings
# dataframes. How many unique user ids are present? How many unique movies
# are there?
movies.head()

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [54]:
print('Number of unique movies:',movies['movie_id'].nunique())

Number of unique movies: 1682


In [55]:
movies.shape

(1682, 24)

In [56]:
print('Number of unique movies:',movies['title'].nunique())

Number of unique movies: 1664


There are movies with the same names. We will drop the ones with the same name.

In [57]:
non_unique = movies.groupby('title').filter(lambda x: len(x) > 1)

In [25]:
non_unique

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
245,246,Chasing Amy (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Chasing+Amy+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
265,266,Kull the Conqueror (1997),29-Aug-1997,NaN,http://us.imdb.com/M/title-exact?Kull+the+Conq...,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
267,268,Chasing Amy (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Chasing+Amy+(...,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
296,297,Ulee's Gold (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Ulee%27s+Gold...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
302,303,Ulee's Gold (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Ulee%27s+Gold...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
303,304,Fly Away Home (1996),13-Sep-1996,NaN,http://us.imdb.com/M/title-exact?Fly%20Away%20...,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
304,305,"Ice Storm, The (1997)",01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Ice+Storm%2C+...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
308,309,Deceiver (1997),01-Jan-1997,NaN,http://us.imdb.com/M/title-exact?Liar+(1997),0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
328,329,Desperate Measures (1998),30-Jan-1998,NaN,http://us.imdb.com/Title?Desperate+Measures+(1...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
347,348,Desperate Measures (1998),30-Jan-1998,NaN,http://us.imdb.com/Title?Desperate+Measures+(1...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [58]:
movies = movies.drop_duplicates(subset='title')

In [59]:
droppeded_movie_ids = [movie_id for movie_id in non_unique['movie_id'].tolist() if movie_id not in movies['movie_id'].tolist()]

We will explore the columns "genre_unknown".

In [60]:
movies.loc[movies['genre_unknown']>0]

,movie_id,title,release_date,video_release_date,imdb_url,genre_unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
266,267,unknown,NaN,NaN,NaN,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1372,1373,Good Morning (1971),4-Feb-1971,NaN,http://us.imdb.com/M/title-exact?Good%20Mornin...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


There are two films with unknown genre and they do not have much infotrmation. We can drop these two rows and then drop the column.

In [61]:
movies.drop(movies.loc[movies['genre_unknown']>0].index, inplace=True)

In [62]:
movies.drop('genre_unknown',axis=1,inplace=True)

In [63]:
genre_cols = [
     "Action", "Adventure", "Animation", "Children", "Comedy",
    "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror",
    "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"
]

In [64]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [65]:
ratings.shape

(100000, 4)

In [66]:
print('Number of unique users:',ratings['user_id'].nunique())

Number of unique users: 943


In [67]:
print('Number of unique movies in ratings dataframe:',ratings['movie_id'].nunique())

Number of unique movies in ratings dataframe: 1682


There are 1664 different movies in movies dataframe. So we will drop the ones that are dropped in movies dataframe.

In [68]:
ratings.drop(ratings.loc[ratings['movie_id'].isin(droppeded_movie_ids)].index, inplace=True)

In [69]:
print('Number of unique movies in ratings dataframe:',ratings['movie_id'].nunique())

Number of unique movies in ratings dataframe: 1664


In [70]:
# Merge movies and ratings dataframes
new_df = pd.merge(movies[['movie_id','title']], ratings[['user_id','movie_id','rating']], on='movie_id')

In [71]:
new_df.drop('movie_id',inplace = True,axis=1)
new_df.head()

,title,user_id,rating
0,Toy Story (1995),308,4
1,Toy Story (1995),287,5
2,Toy Story (1995),148,4
3,Toy Story (1995),280,4
4,Toy Story (1995),66,3


As mentioned in the introduction, content-Based Filtering is a recommendation engine approach that focuses on the attributes or features of items (products, movies, music, articles, etc.) and leverages these features to make personalized recommendations. The underlying idea is to match the characteristics of items with the preferences of users to suggest items that align with their interests. Content-based filtering is particularly useful when explicit user-item interactions (e.g., ratings or purchases) are sparse or unavailable.

**Key Steps in Content-Based Filtering:**

1. **Feature Extraction:**
   - For each item, relevant features are extracted. These features are typically descriptive attributes that can be represented numerically, such as genre, director, actors, author, publication date, and keywords.
   - In the case of text-based items, natural language processing techniques may be used to extract features like TF-IDF (Term Frequency-Inverse Document Frequency) scores.

2. **User Profile Creation:**
   - A user profile is created based on the items they have interacted with in the past. The user profile contains the weighted importance of features based on their interactions.
   - For example, if a user has watched several action movies, the action genre feature would receive a higher weight in their profile.

3. **Similarity Calculation:**
   - The similarity between items or between items and the user profile is calculated using similarity metrics like cosine similarity, Euclidean distance, or Pearson correlation.
   - Cosine similarity is commonly used as it measures the cosine of the angle between two vectors, which represents their similarity.

4. **Recommendation:**
   - Items that are most similar to the user profile are recommended to the user. These are items whose features have the highest similarity scores with the user profile.
   - The recommended items are presented as a list sorted by their similarity scores.

**Advantages of Content-Based Filtering:**
1. **No Cold-Start Problem:** Content-based filtering can make recommendations even for new users with no historical interactions because it relies on item features rather than user history.

2. **User Independence:** The recommendations are based solely on the features of items and do not require knowledge of other users' preferences or behavior.

3. **Transparency:** Content-based recommendations are interpretable, as they depend on the features of items, making it easier for users to understand why specific items are recommended.

4. **Serendipity:** Content-based filtering can recommend items with characteristics not seen before by the user, leading to serendipitous discoveries.

5. **Diversity in Recommendations:** The method can offer diverse recommendations since it suggests items with different feature combinations.

**Limitations of Content-Based Filtering:**
1. **Limited Discovery:** Content-based filtering may struggle to recommend items outside the scope of users' historical interactions or interests.

2. **Over-Specialization:** Users may receive recommendations that are too similar to their previous choices, leading to a lack of exposure to new item categories.

3. **Dependency on Feature Quality:** The quality and relevance of item features significantly influence the quality of recommendations.

4. **Limited for Cold Items:** Content-based filtering can struggle to recommend new items with limited feature information.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return content-based recommendations for this user. Here are steps you can take:

  A. Get the user's rated movies

  B. Create a TF-IDF matrix using movie genres. Note, this can be extracted from the `movies` dataframe.

  C. Compute the cosine similarity between movie genres. Use the [cosine_similarity](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) function.

  D. Get the indices of similar movies to those rated by the user based on cosine similarity. Keep only the top 5.

  E. Remove duplicates and movies already rated by the user.

In [72]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity
tfidf_trans = TfidfTransformer()

# Content-Based Filtering using Movie Genres
def content_based_recommendation(user_id, df):
  # Get the user's rated movies
  user_rated_movies = df.loc[df['user_id'] == user_id,'title'].tolist()

  # Create a TF-IDF matrix using movie genres
  genre_df = movies[genre_cols]
  genre_df.head()
  tfidf_matrix = tfidf_trans.fit_transform(genre_df)

  # Compute the cosine similarity between movie genres
  cosine_sim_matrix = cosine_similarity(tfidf_matrix)
  # Convert the cosine similarity matrix to a DataFrame
  cosine_sim_df = pd.DataFrame(cosine_sim_matrix, index=movies['title'], columns=movies['title'])
  recommendations = []
  for movie in user_rated_movies:
    # Get the indices of similar movies to those rated by the user based on cosine similarity
    similar_movies_scores = cosine_sim_df.loc[movie]
    similar_movies_indices_sorted = similar_movies_scores.sort_values(ascending=False)
    similar_movies_indices = similar_movies_indices_sorted.head(5).index.tolist()
    recommendations.extend(similar_movies_indices)

  # Remove duplicates and movies already rated by the user
  recommendations = [movie for movie in recommendations if movie not in user_rated_movies]
  unique_recommendations = list(set(recommendations))

  return unique_recommendations

In [73]:
len(content_based_recommendation(2, new_df))

132

The key idea behind collaborative filtering is that users who have agreed in the past will likely agree in the future. Instead of relying on item attributes or user profiles, collaborative filtering identifies patterns of user behavior and item preferences from the interactions present in the data.

**Types of Collaborative Filtering:**
There are two main types of collaborative filtering:

**Collaborative Filtering Process:**
The collaborative filtering process typically involves the following steps:

1. **Data Collection:**
   - Gather data on user-item interactions, such as movie ratings, product purchases, or article clicks.

2. **User-Item Matrix:**
   - Organize the data into a user-item matrix, where rows represent users, columns represent items, and the entries contain the users' interactions (e.g., ratings).

3. **Similarity Calculation:**
   - Calculate the similarity between users or items using similarity metrics such as cosine similarity, Pearson correlation, or Jaccard similarity.
   - For user-based collaborative filtering, user similarities are calculated, and for item-based collaborative filtering, item similarities are calculated.

4. **Neighborhood Selection:**
   - For each user or item, select the most similar users or items as the neighborhood.
   - The size of the neighborhood (the number of similar users or items to consider) is an important parameter to control the system's behavior.

5. **Prediction Generation:**
   - Predict the ratings for items that the target user has not yet interacted with by combining the ratings of neighboring users or items.

6. **Recommendation Generation:**
   - Recommend items with the highest predicted ratings to the target user.

**Advantages of Collaborative Filtering using User-Item Interactions:**
- Collaborative filtering is based solely on user interactions and does not require knowledge of item attributes, making it useful for cases where item data is sparse or unavailable.
- It can provide serendipitous recommendations, suggesting items that users may not have discovered on their own.
- Collaborative filtering can be applied in various domains, including e-commerce, music, movie, and content recommendations.

**Limitations of Collaborative Filtering:**
- The cold-start problem: Collaborative filtering struggles to recommend to new users or items with no or limited interaction history.
- It may suffer from sparsity when data is limited or when users have only interacted with a small subset of items.
- Scalability issues can arise with large datasets and an increasing number of users or items.

Here is your task:

1. Write a function that takes in a user id and the dataframe you created before that contains 'user_id', 'title', and 'rating'. The function should return collaborative filtering recommendations for this user based on a user-item interaction matrix. Here are steps you can take:

  A. Create the user-item matrix using Pandas' [pivot_table](https://pandas.pydata.org/docs/reference/api/pandas.pivot_table.html).

  B. Fill missing values with zeros in this matrix.

  C. Calculate user-user similarity matrix using cosine similarity.

  D. Get the array of similarity scores of the target user with all other users from the similarity matrix.

  E. Extract, say the the top 5 most similar users (excluding the target user).

  F. Generate movie recommendations based on the most similar users.

  G. Remove duplicate movies recommendations.

In [74]:
# Collaborative Filtering using User-Item Interactions
def collaborative_filtering_recommendation(user_id, df):
  # Create the user-item matrix
  user_item_matrix = df.pivot_table(index='user_id', columns='title', values='rating')
  # Fill missing values with 0 (indicating no rating)
  #user_item_matrix.sub(user_item_matrix.mean(axis=1), axis=0)
  user_item_matrix.fillna(0, inplace=True)

  # Calculate user-user similarity matrix using cosine similarity
  user_similarity_matrix = cosine_similarity(user_item_matrix)
  # Convert the similarity matrix to a DataFrame
  user_similarity_df = pd.DataFrame(user_similarity_matrix, index=user_item_matrix.index, columns=user_item_matrix.index)

  # Get the similarity scores of the target user with all other users
  target_user_similarity = user_similarity_df.loc[user_id,:]
  # Find the top N most similar users (excluding the target user)
  top_similar_users = target_user_similarity.sort_values(ascending=False)[1:6]
  # Generate movie recommendations based on the most similar users
  recommendations = []
  for user_id in top_similar_users.index:
    # Get the user's rated movies
    mask = user_item_matrix.loc[user_id] != 0
    recommendations.extend(user_item_matrix.loc[user_id,mask].index.tolist())

  # Remove duplicates from recommendations
  unique_recommendations = list(set(recommendations))

  return  unique_recommendations

In [75]:
len(collaborative_filtering_recommendation(2, new_df))

168

Now, test your recommendations engines! Select a few user ids and generate recommendations using both functions you've written. Are the recommendations similar? Do the recommendations make sense?

In [ ]:
# Test the recommendation engines

In [76]:
def user_results(user_id,df):
  user_rated_movies = df.loc[df['user_id'] == user_id,'title'].tolist()
  print("User rated movies \n",user_rated_movies)
  print("Content based recommendations \n",content_based_recommendation(user_id, df))
  print("Collaborative filtering recommendations \n",collaborative_filtering_recommendation(user_id, df))

In [77]:
user_id = 2
user_results(user_id,new_df)

User rated movies 
 ['Toy Story (1995)', 'Richard III (1995)', 'Mighty Aphrodite (1995)', 'Postino, Il (1994)', "Antonia's Line (1995)", 'Birdcage, The (1996)', 'Star Wars (1977)', 'Fargo (1996)', 'Truth About Cats & Dogs, The (1996)', 'Godfather, The (1972)', 'Jerry Maguire (1996)', 'Kolya (1996)', 'Shall We Dance? (1996)', "My Best Friend's Wedding (1997)", 'Men in Black (1997)', 'Contact (1997)', 'Full Monty, The (1997)', 'Good Will Hunting (1997)', 'Heat (1995)', 'Sabrina (1995)', 'Sense and Sensibility (1995)', 'Leaving Las Vegas (1995)', 'Restoration (1995)', 'Bed of Roses (1996)', 'Once Upon a Time... When We Were Colored (1995)', 'Up Close and Personal (1996)', 'River Wild, The (1994)', 'Time to Kill, A (1996)', 'Emma (1996)', 'Tin Cup (1996)', 'Secrets & Lies (1996)', 'English Patient, The (1996)', "Marvin's Room (1996)", 'Scream (1996)', 'Evita (1996)', 'Fierce Creatures (1997)', 'Absolute Power (1997)', 'Rosewood (1997)', 'Donnie Brasco (1997)', 'Liar Liar (1997)', 'Breakdow

In [80]:
user_id = 30
user_results(user_id,new_df)

User rated movies 
 ['GoldenEye (1995)', 'Twelve Monkeys (1995)', 'Apollo 13 (1995)', 'Batman Forever (1995)', 'Star Wars (1977)', 'Forrest Gump (1994)', 'Jurassic Park (1993)', '2001: A Space Odyssey (1968)', 'Top Gun (1986)', 'Abyss, The (1989)', 'Empire Strikes Back, The (1980)', 'Raiders of the Lost Ark (1981)', 'Return of the Jedi (1983)', 'Batman Returns (1992)', 'Kolya (1996)', 'Lost World: Jurassic Park, The (1997)', "My Best Friend's Wedding (1997)", 'Men in Black (1997)', 'Contact (1997)', 'George of the Jungle (1997)', 'English Patient, The (1996)', 'Evita (1996)', 'Liar Liar (1997)', 'In & Out (1997)', 'Fly Away Home (1996)', 'Titanic (1997)', 'Apt Pupil (1998)', 'Everyone Says I Love You (1996)', 'Mother (1996)', 'Batman (1989)', 'Butch Cassidy and the Sundance Kid (1969)', 'Shine (1996)', 'Anastasia (1997)', 'Mouse Hunt (1997)', 'Volcano (1997)', 'Rocket Man (1997)', 'Leave It to Beaver (1997)', 'Tomorrow Never Dies (1997)', 'Dumb & Dumber (1994)', 'Picture Perfect (1997)

When we check the movies rated by the user and the recommended movies it looks they are similar type of movies. It looks the recommender systems do reasonable jobs.

If we look at the intersection of the movies:

In [82]:
def intersection_movies(content_based, collaborative):
  intersection = [movie for movie in collaborative if movie in content_based]
  return intersection

In [84]:
content_based  = content_based_recommendation(user_id, new_df)
collaborative = collaborative_filtering_recommendation(user_id, new_df)
intersection_movies(content_based, collaborative)

['Casablanca (1942)',
 'Until the End of the World (Bis ans Ende der Welt) (1991)',
 'FairyTale: A True Story (1997)',
 'Saint, The (1997)',
 'Man in the Iron Mask, The (1998)',
 'Star Trek: First Contact (1996)',
 'Starship Troopers (1997)',
 'Rock, The (1996)']

In [85]:
len(content_based)

101

In [86]:
len(collaborative)

109

The intersection set is small considering the sizes of recommended movies.